In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/ubuntu/varios/skforecast'

In [2]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn


import numpy as np
import pandas as pd
import time
import warnings
import logging
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
%load_ext pyinstrument 

## Information system and libraries

In [3]:
# Versions
# ==============================================================================
print(f"Python version: {platform.python_version()}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"skforecast version: {skforecast.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scipy version: {scipy.__version__}")
print(f"psutil version: {psutil.__version__}")
print("")

# Computer information
# ==============================================================================
#Computer network name
print(f"Computer network name: {platform.node()}")
#Machine type
print(f"Machine type: {platform.machine()}")
#Processor type
print(f"Processor type: {platform.processor()}")
#Platform type
print(f"Platform type: {platform.platform()}")
#Operating system
print(f"Operating system: {platform.system()}")
#Operating system release
print(f"Operating system release: {platform.release()}")
#Operating system version
print(f"Operating system version: {platform.version()}")
#Physical cores
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
#Logical cores
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.10.9
scikit-learn version: 1.2.1
skforecast version: 0.9.0
pandas version: 2.0.2
numpy version: 1.23.5
scipy version: 1.10.1
psutil version: 5.9.0

Computer network name: ip-10-2-1-218
Machine type: x86_64
Processor type: x86_64
Platform type: Linux-5.15.0-1037-aws-x86_64-with-glibc2.31
Operating system: Linux
Operating system release: 5.15.0-1037-aws
Operating system version: #41~20.04.1-Ubuntu SMP Mon May 22 18:18:00 UTC 2023
Number of physical cores: 4
Number of logical cores: 8


## Data

In [4]:
# Data
# ==============================================================================
n = 1000
rgn = np.random.default_rng(seed=123)
y = pd.Series(rgn.random(size=(n)), name="y")
exog = pd.DataFrame(rgn.random(size=(n, 10)))
exog.columns = [f"col_{i}" for i in range(exog.shape[1])]
y_train = y[:-int(n/2)]
display(y.head())
display(exog.head())   

0    0.682352
1    0.053821
2    0.220360
3    0.184372
4    0.175906
Name: y, dtype: float64

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
0,0.461518,0.189507,0.579512,0.939047,0.299072,0.460476,0.057565,0.085302,0.199445,0.630164
1,0.272450,0.949816,0.416202,0.624220,0.565506,0.497016,0.399685,0.446142,0.996017,0.092076
2,0.966146,0.253428,0.798048,0.230935,0.391439,0.712525,0.924496,0.436304,0.609645,0.347455
3,0.209449,0.940346,0.939049,0.976035,0.978222,0.902929,0.560702,0.309192,0.391578,0.292629
4,0.317627,0.766355,0.939779,0.569968,0.713253,0.274080,0.895022,0.530589,0.662251,0.864017


## Profiling ForecasterAutoreg

In [5]:
warnings.filterwarnings("ignore")

In [6]:
# Forecaster
# ==============================================================================
print("-----------------")
print("ForecasterAutoreg")
print("-----------------")


regressors = [
    Ridge(random_state=77),
    LGBMRegressor(random_state=77, n_jobs=1),
    LGBMRegressor(random_state=77, n_jobs=-1),
    # HistGradientBoostingRegressor(random_state=77),
]
param_grids = [
    {'alpha': [0.1, 1, 10]},
    {'n_estimators': [50, 200], 'max_depth': [5, 10]},
    {'n_estimators': [50, 200], 'max_depth': [5, 10]},
    # {'max_iter': [50, 200], 'max_depth': [5, 10, 15]},
]
lags_grid = [50, 50, 50]

elapsed_times = []

for regressor, param_grid in zip(regressors, param_grids):
    print("")
    print(regressor, param_grid)
    print("")
    forecaster = ForecasterAutoreg(regressor=regressor, lags=50)
    
    # Profiling fit
    # ==============================================================================
    print("Profiling fit")
    start = time.time()
    forecaster.fit(y=y, exog=exog)
    end = time.time()
    elapsed_times.append(end - start)
    
    # Profiling create_train_X_y
    # ==============================================================================
    print("Profiling create_train_X_y")
    start = time.time()
    _ = forecaster.create_train_X_y(y=y, exog=exog)
    end = time.time()
    elapsed_times.append(end - start)

    # Profiling backtesting refit and parallel
    # ==============================================================================
    print("Profiling backtesting refit and parallel")
    start = time.time()
    metric, backtest_predictions = backtesting_forecaster(
                                forecaster          = forecaster,
                                    y                   = y,
                                    exog                = exog,
                                    initial_train_size  = len(y_train),
                                    fixed_train_size    = False,
                                    steps               = 24,
                                    metric              = 'mean_squared_error',
                                    refit               = True,
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    verbose             = False,
                                    show_progress       = False,
                                    n_jobs              = -1
                            )
    end = time.time()
    elapsed_times.append(end - start)
    
    # Profiling backtesting refit and no parallel
    # ==============================================================================
    print("Profiling backtesting refit and no parallel")
    start = time.time()
    metric, backtest_predictions = backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = exog,
                                    initial_train_size  = len(y_train),
                                    fixed_train_size    = False,
                                    steps               = 24,
                                    metric              = 'mean_squared_error',
                                    refit               = True,
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    verbose             = False,
                                    show_progress       = False,
                                    n_jobs              = 1
                            )
    end = time.time()
    elapsed_times.append(end - start)
    
    # Profiling backtesting no refit and parallel
    # ==============================================================================
    print("Profiling backtesting no refit and parallel")
    start = time.time()
    metric, backtest_predictions = backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = exog,
                                    initial_train_size  = len(y_train),
                                    fixed_train_size    = False,
                                    steps               = 24,
                                    metric              = 'mean_squared_error',
                                    refit               = False,
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    verbose             = False,
                                    show_progress       = False,
                                    n_jobs              = -1
                            )
    end = time.time()
    elapsed_times.append(end - start)
    
    # Profiling backtesting no refit and no parallel
    # ==============================================================================
    print("Profiling backtesting no refit and no parallel")
    start = time.time()
    metric, backtest_predictions = backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = exog,
                                    initial_train_size  = len(y_train),
                                    fixed_train_size    = False,
                                    steps               = 24,
                                    metric              = 'mean_squared_error',
                                    refit               = False,
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    verbose             = False,
                                    show_progress       = False,
                                    n_jobs              = 1
                            )
    end = time.time()
    elapsed_times.append(end - start)    
    
    # gridSearch refit parallel
    # ==============================================================================
    print("Profiling gridSearch refit parallel")
    start = time.time()
    results_grid = grid_search_forecaster(
                    forecaster          = forecaster,
                    y                   = y,
                    exog                = exog,
                    initial_train_size  = len(y_train),
                    steps               = 24,
                    param_grid         = param_grid,
                    lags_grid          = lags_grid,
                    refit              = True,
                    metric             = 'mean_squared_error',
                    fixed_train_size   = False,
                    return_best        = False,
                    verbose            = False,
                    show_progress      = False,
                    n_jobs             = -1
            )
    end = time.time()
    elapsed_times.append(end - start)
    
    # GridSearch refit no parallel
    # ==============================================================================
    print("Profiling gridSearch refit no parallel")
    start = time.time()
    results_grid = grid_search_forecaster(
                    forecaster          = forecaster,
                    y                   = y,
                    exog                = exog,
                    initial_train_size  = len(y_train),
                    steps               = 24,
                    param_grid         = param_grid,
                    lags_grid          = lags_grid,
                    refit              = True,
                    metric             = 'mean_squared_error',
                    fixed_train_size   = False,
                    return_best        = False,
                    verbose            = False,
                    show_progress      = False,
                    n_jobs             = 1
            )
    end = time.time()
    elapsed_times.append(end - start)    

-----------------
ForecasterAutoreg
-----------------

Ridge(random_state=77) {'alpha': [0.1, 1, 10]}

Profiling fit
Profiling create_train_X_y
Profiling backtesting refit and parallel


Profiling backtesting refit and no parallel
Profiling backtesting no refit and parallel
Profiling backtesting no refit and no parallel
Profiling gridSearch refit parallel
Number of models compared: 9.
Profiling gridSearch refit no parallel
Number of models compared: 9.

LGBMRegressor(n_jobs=1, random_state=77) {'n_estimators': [50, 200], 'max_depth': [5, 10]}

Profiling fit
Profiling create_train_X_y
Profiling backtesting refit and parallel
Profiling backtesting refit and no parallel
Profiling backtesting no refit and parallel
Profiling backtesting no refit and no parallel
Profiling gridSearch refit parallel
Number of models compared: 12.
Profiling gridSearch refit no parallel
Number of models compared: 12.

LGBMRegressor(random_state=77) {'n_estimators': [50, 200], 'max_depth': [5, 10]}

Profiling fit
Profiling create_train_X_y
Profiling backtesting refit and parallel
Profiling backtesting refit and no parallel
Profiling backtesting no refit and parallel
Profiling backtesting no refit

In [ ]:
methods = [
    "fit",
    "create_train_X_y",
    "backtest_refit_parallel",
    "backtest_refit_noparallel",
    "backtest_no_refit_parallel",
    "backtest_no_refit_noparallel",
    "gridSearch_refit_parallel",
    "gridSearch_refit_noparallel"
]

results = pd.DataFrame({
     "regressor": np.repeat(np.array([str(regressor) for regressor in regressors]), len(methods)),
     "method": np.tile(methods, len(regressors)),
     "elapsed_time": elapsed_times
})

results['parallel'] = results.method.str.contains("_parallel")
results['method'] = results.method.str.replace("_parallel", "")
results['method'] = results.method.str.replace("_noparallel", "")
results

results.sort_values(by=["regressor", "method", "parallel"])

,regressor,method,elapsed_time,parallel
0,Ridge(random_state=77),fit,0.009634,False
1,Ridge(random_state=77),create_train_X_y,0.003325,False
2,Ridge(random_state=77),backtest_refit,0.162785,True
3,Ridge(random_state=77),backtest_refit,0.410622,False
4,Ridge(random_state=77),backtest_no_refit,0.129821,True
5,Ridge(random_state=77),backtest_no_refit,0.127650,False
6,Ridge(random_state=77),gridSearch_refit,0.510144,True
7,Ridge(random_state=77),gridSearch_refit,2.798442,False
8,"LGBMRegressor(n_jobs=1, random_state=77)",fit,0.522166,False
9,"LGBMRegressor(n_jobs=1, random_state=77)",create_train_X_y,0.002484,False


In [ ]:
regressor =LGBMRegressor(random_state=77, n_jobs=-1)
forecaster = ForecasterAutoreg(regressor=regressor, lags=50)

print("Profiling backtesting refit and parallel")
print(regressor)
start = time.time()
metric, backtest_predictions = backtesting_forecaster(
                            forecaster          = forecaster,
                                y                   = y,
                                exog                = exog,
                                initial_train_size  = len(y_train),
                                fixed_train_size    = False,
                                steps               = 24,
                                metric              = 'mean_squared_error',
                                refit               = True,
                                interval            = None,
                                n_boot              = 500,
                                random_state        = 123,
                                in_sample_residuals = True,
                                verbose             = False,
                                show_progress       = False,
                                n_jobs              = -1
                        )
end = time.time()
print(end - start)

Profiling backtesting refit and parallel
LGBMRegressor(random_state=77)
1.6172480583190918


In [ ]:
regressor =LGBMRegressor(random_state=77, n_jobs=-1)
forecaster = ForecasterAutoreg(regressor=regressor, lags=50)
print("Profiling gridSearch refit parallel")
print(regressor)
param_grid = [
    {'n_estimators': [50, 200], 'max_depth': [5, 10]}
]
lags_grid = [50, 50, 50]
start = time.time()
results_grid = grid_search_forecaster(
                forecaster          = forecaster,
                y                   = y,
                exog                = exog,
                initial_train_size  = len(y_train),
                steps               = 24,
                param_grid         = param_grid,
                lags_grid          = lags_grid,
                refit              = True,
                metric             = 'mean_squared_error',
                fixed_train_size   = False,
                return_best        = False,
                verbose            = False,
                show_progress      = False,
                n_jobs             = -1
        )
end = time.time()
print(end - start)

Profiling gridSearch refit parallel
LGBMRegressor(random_state=77)
Number of models compared: 12.
16.608192205429077
